In [1]:
import pandas as pd

from oof_unet.data import OofUniqueMachineDataset, VolumeBasedBatchSampler
from torch.utils.data import DataLoader


In [2]:
df = pd.read_csv('../csv/Dataset001_final.csv')
df_train = df[df['split'] == 'Train']
df_train = df_train.reset_index(drop=True)

list_train = df_train['id_dosemap'].tolist()


In [3]:
from sklearn.model_selection import KFold

In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [5]:
for fold, (_, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold'] = f'Val_{fold}'

In [6]:
df_train.to_csv('Dataset001_final_train.csv', index=False)

In [7]:
df_test = df[df['split'] == 'Test']
df_test = df_test.reset_index(drop=True)

df_test.to_csv('Dataset001_final_test.csv', index=False)

In [8]:
train_dataset = OofUniqueMachineDataset(list_dosemaps=list_train, split='Train')
train_sampler = VolumeBasedBatchSampler(dataset=train_dataset, n_units=20, unit_volume=10_000_000)

dataloader = DataLoader(train_dataset, batch_sampler=train_sampler, num_workers=4)


In [9]:
i = 0

for batch in dataloader:

    print(batch["in_field"].shape)

    i += 1

    if i == 10:
        break

torch.Size([11, 160, 192, 576])
torch.Size([5, 160, 192, 576])
torch.Size([6, 208, 208, 720])
torch.Size([6, 208, 208, 720])


KeyboardInterrupt: 

In [15]:
import torch

In [16]:
in_field = batch["in_field"].unsqueeze(1)
mask = batch["mask"].unsqueeze(1)
inputs = torch.cat([in_field, mask], dim=1)

targets = batch["out_field"].unsqueeze(1)

In [17]:
inputs.shape

torch.Size([6, 2, 208, 208, 720])

In [18]:
targets.shape


torch.Size([6, 1, 208, 208, 720])